Comienzo importando los módulos .py que armé:

In [4]:
import sys
import os
import pprint

#Línea para generar acceso a los módulos
sys.path.append(os.path.join(os.getcwd(), "src"))

from evaluar import resultados_equipo, mejor_equipo_ronda
from acumular import inicializar_acumulados, actualizar_acumulados, marcar_mejor_equipo


A continuación, implemento las funciones para resolver la actividad

In [5]:
evaluaciones = [
 # Ronda 1
 {
 'EquipoA': {'innovacion': 2, 'presentacion': 1, 'errores': True},
 'EquipoB': {'innovacion': 1, 'presentacion': 0, 'errores': False},
 'EquipoC': {'innovacion': 1, 'presentacion': 2, 'errores': True},
 'EquipoD': {'innovacion': 0, 'presentacion': 1, 'errores': False},
 'EquipoE': {'innovacion': 1, 'presentacion': 1, 'errores': False}
 },
 # Ronda 2
 {
 'EquipoA': {'innovacion': 0, 'presentacion': 2, 'errores': False},
 'EquipoB': {'innovacion': 2, 'presentacion': 0, 'errores': True},
 'EquipoC': {'innovacion': 1, 'presentacion': 1, 'errores': False},
 'EquipoD': {'innovacion': 2, 'presentacion': 1, 'errores': True},
 'EquipoE': {'innovacion': 0, 'presentacion': 1, 'errores': False}
 },
 # Ronda 3
 {
 'EquipoA': {'innovacion': 3, 'presentacion': 2, 'errores': False},
 'EquipoB': {'innovacion': 1, 'presentacion': 1, 'errores': True},
 'EquipoC': {'innovacion': 2, 'presentacion': 0, 'errores': False},
 'EquipoD': {'innovacion': 1, 'presentacion': 3, 'errores': True},
 'EquipoE': {'innovacion': 2, 'presentacion': 2, 'errores': False}
 },
 # Ronda 4
 {
 'EquipoA': {'innovacion': 1, 'presentacion': 3, 'errores': True},
 'EquipoB': {'innovacion': 2, 'presentacion': 2, 'errores': False},
 'EquipoC': {'innovacion': 3, 'presentacion': 1, 'errores': False},
 'EquipoD': {'innovacion': 0, 'presentacion': 2, 'errores': True},
 'EquipoE': {'innovacion': 2, 'presentacion': 0, 'errores': False}
 },
 # Ronda 5
 {
 'EquipoA': {'innovacion': 2, 'presentacion': 2, 'errores': False},
 'EquipoB': {'innovacion': 1, 'presentacion': 3, 'errores': True},
 'EquipoC': {'innovacion': 0, 'presentacion': 2, 'errores': False},
 'EquipoD': {'innovacion': 3, 'presentacion': 1, 'errores': False},
 'EquipoE': {'innovacion': 2, 'presentacion': 3, 'errores': True}
 }
]

acumulados = inicializar_acumulados(evaluaciones)        #creo el diccionario de acumulados
i = 1                   # CAMBIAR, es para la impresión de la ronda
for ronda in evaluaciones:
    dict_ronda = ronda.copy()  # Genero una copia de la ronda para no modificar "evaluaciones"
    mejor_puntaje = -1         # Para ser superado fácilmente
    mejor_nombre = None        # Para ser superado fácilmente
    
    for nombre_equipo, datos_equipo in ronda.items():
        dict_equipo = datos_equipo.copy()             # Genero copia del equipo para no modificar original
        dict_equipo = resultados_equipo(dict_equipo)  # Agrego puntaje en la copia

        # Busco el resultado actualizado de mejor equipo
        mejor_nombre, mejor_puntaje = mejor_equipo_ronda(nombre_equipo, dict_equipo, mejor_puntaje, mejor_nombre)
        
        dict_ronda[nombre_equipo] = dict_equipo     # Agrego copia equipo a copia de ronda

        actualizar_acumulados(acumulados, dict_equipo, nombre_equipo) # Por ahora actualizo sin "mejores"
    
    equipos_ordenados = sorted(
        dict_ronda.items(),
        key=lambda item: item[1]['puntaje'],                #lambda, FUNCIONES ANÓNIMAS (video)
        reverse=True
    )
    dict_ronda = {
    nombre: {**datos, 'mejores': nombre == mejor_nombre}              #dict comprehension agregandole el dato "mejor", con KWARGS (explicar en video),"podría haber hecho una función y usar map peroes mejor"
    for nombre, datos in equipos_ordenados
    }    

    
    # Agregarle a acumulados lo de mejores por fuera del bucle de equipos, para ahorrar un for
    marcar_mejor_equipo(acumulados, mejor_nombre)
   
    print(f"\n🏁Ronda {i} : {ronda}")
    print("-" * 50)
    print(f"{'Equipo':<10} {'Innov':<6} {'Pres':<6} {'Errores':<8} {'Puntaje':<8} {'Mejor':<6}")
    print("-" * 50)

    for nombre, datos in dict_ronda.items():
        print(f"{nombre:<10} {datos['innovacion']:<6} {datos['presentacion']:<6} {str(datos['errores']):<8} {datos['puntaje']:<8} {str(datos['mejores']):<6}")

    i += 1

print("\n📊 Resultados Finales")
print("-" * 50)
print(f"{'Equipo':<10} {'Puntaje':<8} {'Mejores Rondas':<15}")
print("-" * 50)

# Ordenar los acumulados por puntaje descendente
acumulados_ordenados = sorted(
    acumulados.items(),
    key=lambda item: item[1]['puntaje'],
    reverse=True
)

for nombre, datos in acumulados_ordenados:
    print(f"{nombre:<10} {datos['puntaje']:<8} {datos['mejores']:<15}")
        
    



🏁Ronda 1 : {'EquipoA': {'innovacion': 2, 'presentacion': 1, 'errores': True}, 'EquipoB': {'innovacion': 1, 'presentacion': 0, 'errores': False}, 'EquipoC': {'innovacion': 1, 'presentacion': 2, 'errores': True}, 'EquipoD': {'innovacion': 0, 'presentacion': 1, 'errores': False}, 'EquipoE': {'innovacion': 1, 'presentacion': 1, 'errores': False}}
--------------------------------------------------
Equipo     Innov  Pres   Errores  Puntaje  Mejor 
--------------------------------------------------
EquipoA    2      1      True     6        True  
EquipoC    1      2      True     4        False 
EquipoE    1      1      False    4        False 
EquipoB    1      0      False    3        False 
EquipoD    0      1      False    1        False 

🏁Ronda 2 : {'EquipoA': {'innovacion': 0, 'presentacion': 2, 'errores': False}, 'EquipoB': {'innovacion': 2, 'presentacion': 0, 'errores': True}, 'EquipoC': {'innovacion': 1, 'presentacion': 1, 'errores': False}, 'EquipoD': {'innovacion': 2, 'presentac